In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
custom_transform= transforms.Compose([ transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_set=datasets.FashionMNIST('./data', train=True, download=True,transform=custom_transform)
test_set=datasets.FashionMNIST('./data', train=False, transform=custom_transform)

In [4]:
loader = torch.utils.data.DataLoader(train_set, batch_size = 64)

In [5]:
def get_data_loader(data_set = True):
    if data_set == True:
        loader = torch.utils.data.DataLoader(train_set, batch_size = 64)
    elif data_set == False:
        loader = torch.utils.data.DataLoader(test_set, batch_size = 64)
    else:
        loader = "train set = True, test set = False"

    return loader

In [19]:
train_loader = get_data_loader(True)
test_loader = get_data_loader(False)
print(type(train_loader))
print(train_loader.dataset)

<class 'torch.utils.data.dataloader.DataLoader'>
Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [20]:
model = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 128), # 28x28 = 784 (input), 128 (output)
                    nn.ReLU(),
                    nn.Linear(128, 64), # 128 (input), 64 (output)
                    nn.ReLU(),
                    nn.Linear(64, 10) # 64 (input), 10 (output)
                    )

print(model)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=10, bias=True)
)


In [27]:
def train_model(model, train_loader, criterion, T):
    for epoch in range(T):
        # Set the model to train mode
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        model.train()
        
        # Initialize the loss and accuracy for this epoch
        epoch_loss = 0.0
        epoch_correct = 0
        
        for i, (images, labels) in enumerate(train_loader):
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            # Update the epoch loss and accuracy
            epoch_loss += loss.item() * images.shape[0]
            _, predictions = torch.max(outputs.data, 1)
            epoch_correct += (predictions == labels).sum().item()
        
        # Print the training status for this epoch
        epoch_loss /= len(train_loader.dataset)
        epoch_acc = 100.0 * epoch_correct / len(train_loader.dataset)
        print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%) Loss: {:.3f}'.format(
            epoch, epoch_correct, len(train_loader.dataset), epoch_acc, epoch_loss))
        
T = 5
criterion = nn.CrossEntropyLoss()
train_model(model, train_loader, criterion, T)

Train Epoch: 0 Accuracy: 41692/60000 (69.49%) Loss: 0.901
Train Epoch: 1 Accuracy: 49252/60000 (82.09%) Loss: 0.512
Train Epoch: 2 Accuracy: 50413/60000 (84.02%) Loss: 0.454
Train Epoch: 3 Accuracy: 51043/60000 (85.07%) Loss: 0.423
Train Epoch: 4 Accuracy: 51482/60000 (85.80%) Loss: 0.401


In [30]:
def evaluate_model(model, test_loader, criterion, show_loss=True):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            test_loss += criterion(outputs, labels).item() * images.shape[0]
            _, predictions = torch.max(outputs.data, 1)
            correct += (predictions == labels).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100.0 * correct / len(test_loader.dataset)

    if show_loss:
        print('Average loss: {:.4f}'.format(test_loss))
    print('Accuracy: {:.2f}%'.format(test_acc))

evaluate_model(model, test_loader, criterion, show_loss=False)
evaluate_model(model, test_loader, criterion, show_loss=True)

Accuracy: 84.69%
Average loss: 0.4253
Accuracy: 84.69%


In [50]:
def predict_label(model, test_images, index):
    class_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt'
                    ,'Sneaker','Bag','Ankle Boot']
    
    model.eval()
    with torch.no_grad():
        output = model(test_images[index])
        prob = F.softmax(output, dim=1)
        values, indices = prob.topk(3)
        for i in range(3):
            print('{}: {:.2f}%'.format(class_names[indices[0][i]], values[0][i]*100))

test_images, test_labels = next(iter(test_loader))
predict_label(model, test_images, 1)

Pullover: 96.51%
Shirt: 2.71%
Coat: 0.69%
